In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM,Dense,Embedding
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from pickle import dump

In [3]:
from google.colab import files
uploaded = files.upload()


Saving pluto_book_1.txt to pluto_book_1.txt


In [0]:
filename="pluto_book_1.txt"

In [0]:
# opening file
file=open(filename,"r")
text=file.read()


In [0]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
sequence=tokenizer.texts_to_sequences([text])[0]

In [7]:
vocab=len(tokenizer.word_index)+1
vocab

7489

In [0]:
word_sequence=[]
for i in range(1,len(sequence)):
    sequences=sequence[i-1:i+1]
    word_sequence.append(sequences)
word_sequence=np.array(word_sequence)

In [0]:
X=word_sequence[:,0]
Y=word_sequence[:,1]

In [10]:
print("X:",X)
print("Y:",Y)

X: [ 11 973 326 ...  21  23  86]
Y: [ 973  326 4113 ...   23   86  488]


In [0]:
Y=np_utils.to_categorical(Y,num_classes=vocab)

In [13]:
model = Sequential()
model.add(Embedding(vocab, 50))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

W0730 17:34:31.062023 139887548348288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0730 17:34:31.073789 139887548348288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          374450    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 100)         60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 7489)              756389    
Total params: 1,281,739
Trainable params: 1,281,739
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.8MB/s 


In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, Y, batch_size=128, epochs=5)
 
model.save('model.h5')
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Epoch 1/5
119109/119109 [==============================] - 18s 153us/step - loss: 4.0478 - acc: 0.2037
Epoch 2/5
119109/119109 [==============================] - 16s 135us/step - loss: 4.0088 - acc: 0.2033
Epoch 3/5
119109/119109 [==============================] - 16s 135us/step - loss: 3.9995 - acc: 0.2028
Epoch 4/5
119109/119109 [==============================] - 16s 135us/step - loss: 3.9930 - acc: 0.2026
Epoch 5/5
119109/119109 [==============================] - 16s 135us/step - loss: 3.9876 - acc: 0.2028


In [0]:
model_file = drive.CreateFile({'title' : 'model.h5'})
model_file.SetContentFile('model.h5')
model_file.Upload()
drive.CreateFile({'id': model_file.get('id')})


In [25]:
word=input("enter word:")
no_of_next=int(input("enter no. of words you wanna predict:"))
in_word,next_word=word,word
for i in range(no_of_next):
    print(f"prediction:{i+1}")
    out_word=""
    encoded_test=tokenizer.texts_to_sequences([in_word])[0]
    encoded_test=np.array(encoded_test)
    pred=model.predict_classes(encoded_test,verbose=0)
    for word,index in tokenizer.word_index.items():
        if pred == index:
            out_word=word
            break
    in_word=out_word
    print(f"next word is:{out_word}")

enter word:artemis
enter no. of words you wanna predict:2
prediction:1
next word is:and
prediction:2
next word is:the
